# Set up

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd '/content/drive/My Drive/deteccao_sobrepreco/'

/content/drive/My Drive/deteccao_sobrepreco


In [4]:
!ls -lah

total 3.8G
-rw------- 1 root root  42M Sep  6 13:37  amostra-notas.csv
-rw------- 1 root root 200K Mar  1 22:04 'Análise estatística descritiva 3.ipynb'
drwx------ 2 root root 4.0K Sep  6 15:04  candidatos
-rw------- 1 root root 123M Apr 30  2021  consulta_cand_2016.zip
-rw------- 1 root root 162K Mar  1 22:28 'Cópia de Análise estatística descritiva 3.ipynb'
-rw------- 1 root root  78K Mar  1 14:03 'Cópia de levenshtein.ipynb'
-rw------- 1 root root  39K Sep  6 20:51  empresas_cidades2.gexf
-rw------- 1 root root  80K Sep  7 00:48  empresas_cidades3.gexf
-rw------- 1 root root  79K Sep  7 14:12  empresas_cidades4.gexf
-rw------- 1 root root  39K Sep  6 20:26  empresas_cidades.gexf
-rw------- 1 root root  86K Sep  8 17:58  grafos_alimentos_sobrepreco_1.1.gexf
-rw------- 1 root root  83K Sep  8 17:58  grafos_alimentos_sobrepreco_1.5.gexf
-rw------- 1 root root  86K Sep 10 17:51  grafos_alimentos_sobrepreco_1.gexf
-rw------- 1 root root  78K Sep 10 17:44  grafos_alimentos_sobreprec

In [7]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Reading and treating data

In [8]:
notas = pd.read_csv('Notas_Fiscais.csv', sep=',', encoding='latin1', low_memory=False, dtype={
    'Numero': str, 'Data_de_emissao': str, 'Situacao': str, 'Valor_total_da_nota': float,
    'Nome_razao_social_emit': str, 'CPF_CNPJ_emit': str, 'Endereco_emit': str,
    'Bairro_distrito_emit': str, 'CEP_emit': str, 'Municipio_emit': str, 'Nome_razao_social_dest': str,
    'CPF_CNPJ_dest': str, 'Endereco_dest': str, 'Bairro_distrito_dest': str, 'CEP_dest': str,
    'Municipio_dest': str, 'Base_de_Calculo_do_ICMS': float, 'Valor_do_ICMS': float,
    'Base_de_calculo_do_ICMS_substituicao': float, 'Valor_do_ICMS_substituicao': float,
    'Valor_total_dos_produtos': float, 'Valor_do_frete': float, 'Valor_do_seguro': float, 'Valor_desconto': float,
    'Valor_outras_despesas_acessorias': float, 'Valor_do_IPI': float, 'Valor_total_ICMS_UF_dest': float,
    'Valor_total_ICMS_UF_remet': float, 'Valor_BC_ICMS_UF_dest': float, 'Aliquota_interna_UF_dest': float,
    'Aliquota_interestadual_UF_env': float, 'Perc_prov_partilha_UF': float, 'Perc_ICMS_FCP_UF_dest': float,
    'Valor_ICMS_FCP_UF_dest': float, 'Valor_ICMS_partilha_UF_dest': float, 'Valor_ICMS_partilha_UF_remet': float,
    'Nr_item': str, 'Cod_prod': str, 'Descricao_do_Produto_ou_servicos': str, 'NCM_prod': str, 'CST_prod': str,
    'CFOP_prod': str, 'Quant_prod': float, 'Valor_unit_prod': float, 'Valor_total_prod': float,
    'Valor_desconto_item': float, 'BC_ICMS_prod': float, 'Valor_ICMS_prod': float, 'Aliq_ICMS_prod': float,
    'BC_ICMS_ST_prod': float, 'Valor_ICMS_ST_prod': float, 'Aliq_ICMS_ST_prod': float, 'Valor_IPI_prod': float,
    'Aliq_IPI_prod': float, 'Valor_PMC_prod': float, 'Cod_EAN': str, 'Info_Adicional_Item': str,
    'Informacoes_adicionais_do_FISCO': str, 'Informacoes_complementares': str, 'Unid_prod': str,
    'Metrica': str})

notas.head()

,Numero,Data_de_emissao,Situacao,Valor_total_da_nota,Nota_referenciada,Nome_razao_social_emit,CPF_CNPJ_emit,Endereco_emit,Bairro_distrito_emit,CEP_emit,Municipio_emit,Nome_razao_social_dest,CPF_CNPJ_dest,Endereco_dest,Bairro_distrito_dest,CEP_dest,Municipio_dest,Base_de_Calculo_do_ICMS,Valor_do_ICMS,Base_de_calculo_do_ICMS_substituicao,Valor_do_ICMS_substituicao,Valor_total_dos_produtos,Valor_do_frete,Valor_do_seguro,Valor_desconto,Valor_outras_despesas_acessorias,Valor_do_IPI,Valor_total_ICMS_UF_dest,Valor_total_ICMS_UF_remet,Valor_BC_ICMS_UF_dest,Aliquota_interna_UF_dest,Aliquota_interestadual_UF_env,Perc_prov_partilha_UF,Perc_ICMS_FCP_UF_dest,Valor_ICMS_FCP_UF_dest,Valor_ICMS_partilha_UF_dest,Valor_ICMS_partilha_UF_remet,Nr_item,Cod_prod,Descricao_do_Produto_ou_servicos,NCM_prod,CST_prod,CFOP_prod,Quant_prod,Valor_unit_prod,Valor_total_prod,Valor_desconto_item,BC_ICMS_prod,Valor_ICMS_prod,Aliq_ICMS_prod,BC_ICMS_ST_prod,Valor_ICMS_ST_prod,Aliq_ICMS_ST_prod,Valor_IPI_prod,Aliq_IPI_prod,Valor_PMC_prod,Cod_EAN,Info_Adicional_Item,Informacoes_adicionais_do_FISCO,Informacoes_complementares,Unid_prod,Metrica
0,8326,2016-02-04 14:30:38,A,45.99,NaN,LEIA COMERCIAL DE LIVROS E MAGAZINE LTDA,11895653000100,AV. PRESIDENTE JOAO PESSOA,CENTRO,58900000,CAJAZEIRAS,ADMINISTRACAO DO SISTEMA PREVIDENCIARIO,12724621000105,"R JOSE GOMES,S/N",FRANCISCO MESAO FRANCISCO,58900000,Nazarezinho,0.0,0.0,0.0,0.0,54.1,0.0,0.0,8.11,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,085213,GRAMPO PLAST DELLO FIX BC - 0301.E0020.2 - DELO,39261000,0,5929,1.0,9.95,9.95,1.49,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,7897832805223,NaN,NaN,Referente o Cupom :(050213) Do ECF: (0002),UND,0.0240259227060776
1,8326,2016-02-04 14:30:38,A,45.99,NaN,LEIA COMERCIAL DE LIVROS E MAGAZINE LTDA,11895653000100,AV. PRESIDENTE JOAO PESSOA,CENTRO,58900000,CAJAZEIRAS,ADMINISTRACAO DO SISTEMA PREVIDENCIARIO,12724621000105,"R JOSE GOMES,S/N",FRANCISCO MESAO FRANCISCO,58900000,Nazarezinho,0.0,0.0,0.0,0.0,54.1,0.0,0.0,8.11,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,088949,LAPISEIRA TECNICA NPY 0.9 - 14088 - MOLIN,96084000,0,5929,1.0,8.30,8.30,1.24,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,7898538299552,NaN,NaN,Referente o Cupom :(050213) Do ECF: (0002),UND,0.117620949844652
2,8326,2016-02-04 14:30:38,A,45.99,NaN,LEIA COMERCIAL DE LIVROS E MAGAZINE LTDA,11895653000100,AV. PRESIDENTE JOAO PESSOA,CENTRO,58900000,CAJAZEIRAS,ADMINISTRACAO DO SISTEMA PREVIDENCIARIO,12724621000105,"R JOSE GOMES,S/N",FRANCISCO MESAO FRANCISCO,58900000,Nazarezinho,0.0,0.0,0.0,0.0,54.1,0.0,0.0,8.11,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,082601,LAPISEIRA TEC JOB 0.7 - 14077 - MOLIN,96084000,0,5929,1.0,6.95,6.95,1.04,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,7898538299460,NaN,NaN,Referente o Cupom :(050213) Do ECF: (0002),UND,0.119840213049268
3,8326,2016-02-04 14:30:38,A,45.99,NaN,LEIA COMERCIAL DE LIVROS E MAGAZINE LTDA,11895653000100,AV. PRESIDENTE JOAO PESSOA,CENTRO,58900000,CAJAZEIRAS,ADMINISTRACAO DO SISTEMA PREVIDENCIARIO,12724621000105,"R JOSE GOMES,S/N",FRANCISCO MESAO FRANCISCO,58900000,Nazarezinho,0.0,0.0,0.0,0.0,54.1,0.0,0.0,8.11,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,083267,MARCA TEXTO FLUO PEPS AMARELO - 742534 - MAPED,96082000,0,5929,1.0,3.70,3.70,0.55,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,3154147425343,NaN,NaN,Referente o Cupom :(050213) Do ECF: (0002),UND,0.101036047606816
4,8326,2016-02-04 14:30:38,A,45.99,NaN,LEIA COMERCIAL DE LIVROS E MAGAZINE LTDA,11895653000100,AV. PRESIDENTE JOAO PESSOA,CENTRO,58900000,CAJAZEIRAS,ADMINISTRACAO DO SISTEMA PREVIDENCIARIO,12724621000105,"R JOSE GOMES,S/N",FRANCISCO MESAO FRANCISCO,58900000,Nazarezinho,0.0,0.0,0.0,0.0,54.1,0.0,0.0,8.11,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,084814,GRAMPEADOR 26/6 T320 12CM 20F PRETO - 665207 ...,84729040,0,5929,1.0,16.50,16.50,2.47,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,7897476665207,NaN,NaN,Referente o Cupom :(050213) Do ECF: (0002),UND,0.209855654840586


In [9]:
notas.columns

Index(['Numero', 'Data_de_emissao', 'Situacao', 'Valor_total_da_nota',
       'Nota_referenciada', 'Nome_razao_social_emit', 'CPF_CNPJ_emit',
       'Endereco_emit', 'Bairro_distrito_emit', 'CEP_emit', 'Municipio_emit',
       'Nome_razao_social_dest', 'CPF_CNPJ_dest', 'Endereco_dest',
       'Bairro_distrito_dest', 'CEP_dest', 'Municipio_dest',
       'Base_de_Calculo_do_ICMS', 'Valor_do_ICMS',
       'Base_de_calculo_do_ICMS_substituicao', 'Valor_do_ICMS_substituicao',
       'Valor_total_dos_produtos', 'Valor_do_frete', 'Valor_do_seguro',
       'Valor_desconto', 'Valor_outras_despesas_acessorias', 'Valor_do_IPI',
       'Valor_total_ICMS_UF_dest', 'Valor_total_ICMS_UF_remet',
       'Valor_BC_ICMS_UF_dest', 'Aliquota_interna_UF_dest',
       'Aliquota_interestadual_UF_env', 'Perc_prov_partilha_UF',
       'Perc_ICMS_FCP_UF_dest', 'Valor_ICMS_FCP_UF_dest',
       'Valor_ICMS_partilha_UF_dest', 'Valor_ICMS_partilha_UF_remet',
       'Nr_item', 'Cod_prod', 'Descricao_do_Produto_ou_ser

In [10]:
notas.rename(columns={'Descricao_do_Produto_ou_servicos':'descricao','Cod_EAN':'ean','NCM_prod':'ncm'},inplace=True)
notas = notas.loc[notas['Situacao'] == 'A']
notas['descricao'] = notas['descricao'].apply(lambda x: str(x).upper())
notas['Municipio_dest'] = notas['Municipio_dest'].apply(lambda x: str(x).upper())
notas['Data_de_emissao'] = notas['Data_de_emissao'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())

In [11]:
cenoura = notas[notas['descricao'] == 'CENOURA'][['Data_de_emissao','Valor_total_da_nota','Municipio_dest','Valor_total_dos_produtos','Valor_desconto','Quant_prod',
                                           'Valor_unit_prod','Valor_total_prod','CPF_CNPJ_emit','Unid_prod','ncm','ean','descricao']].sort_values('Valor_unit_prod',ascending=False)

In [12]:
tomate = notas[notas['descricao'] == 'TOMATE'][['Data_de_emissao','Valor_total_da_nota','Municipio_dest','Valor_total_dos_produtos','Valor_desconto','Quant_prod',
                                           'Valor_unit_prod','Valor_total_prod','CPF_CNPJ_emit','Unid_prod','ncm','ean','descricao']].sort_values('Valor_unit_prod',ascending=False)

In [13]:
cebola = notas[notas['descricao'] == 'CEBOLA'][['Data_de_emissao','Valor_total_da_nota','Municipio_dest','Valor_total_dos_produtos','Valor_desconto','Quant_prod',
                                           'Valor_unit_prod','Valor_total_prod','CPF_CNPJ_emit','Unid_prod','ncm','ean','descricao']].sort_values('Valor_unit_prod',ascending=False)

In [14]:
alho = notas[notas['descricao'] == 'ALHO'][['Data_de_emissao','Valor_total_da_nota','Municipio_dest','Valor_total_dos_produtos','Valor_desconto','Quant_prod',
                                           'Valor_unit_prod','Valor_total_prod','CPF_CNPJ_emit','Unid_prod','ncm','ean','descricao']].sort_values('Valor_unit_prod',ascending=False)

In [15]:
batata_inglesa = notas[notas['descricao'] == 'BATATA INGLESA'][['Data_de_emissao','Valor_total_da_nota','Municipio_dest','Valor_total_dos_produtos','Valor_desconto','Quant_prod',
                                           'Valor_unit_prod','Valor_total_prod','CPF_CNPJ_emit','Unid_prod','ncm','ean','descricao']].sort_values('Valor_unit_prod',ascending=False)

In [17]:
cuscuz = notas[notas['descricao'].str.contains('FLOCOS')][['Data_de_emissao','Valor_total_da_nota','Municipio_dest','Valor_total_dos_produtos','Valor_desconto','Quant_prod',
                                           'Valor_unit_prod','Valor_total_prod','CPF_CNPJ_emit','Unid_prod','ncm','ean','descricao']].sort_values('Valor_unit_prod',ascending=False)
cuscuz = cuscuz[cuscuz['descricao'].str.contains('MILHO')]
cuscuz = cuscuz[cuscuz['descricao'].str.contains('500G')]
cuscuz = cuscuz[~cuscuz['Unid_prod'].str.contains('FD')]
cuscuz = cuscuz[~cuscuz['Unid_prod'].str.contains('1')]
cuscuz = cuscuz[~cuscuz['Unid_prod'].str.contains('KG')]
s = cuscuz.shape[0]
cuscuz = cuscuz[int(0.05*s):int(0.95*s)]

# Function to create a matriz and an indicator of overprice

In [37]:
def sobrepreco_mun(dataframe, produto):
  empresa = {'cnpj':[],'periodo':[],'matriz':[],'valor_total':[],'produto':[]}
  periodos = []

  cnpj = dataframe['CPF_CNPJ_emit'].value_counts().to_frame()
  cnpj = cnpj.loc[cnpj['CPF_CNPJ_emit'] >= 180].index

  for n in cnpj:
    for i in range(1,13):
      start_date = datetime(year=2016, month=i, day=1).date()
      end_date = datetime(year=2016, month=i+1 if i < 12 else i, day=1).date()
      filtro_data = (dataframe['Data_de_emissao'] >= start_date) & (dataframe['Data_de_emissao'] < end_date)
      df = dataframe[(dataframe['CPF_CNPJ_emit']== n) & filtro_data]
      valor_total = df['Valor_total_prod'].sum()

      municipios = df['Municipio_dest'].unique()

      matriz = pd.DataFrame(np.zeros((len(municipios),len(municipios))),columns=municipios)
      matriz['index'] = municipios
      matriz = matriz.set_index('index')
      empresa['cnpj'].append(n)
      empresa['periodo'].append(str(str(i)+'-'+str(i+1)))
      empresa['valor_total'].append(valor_total)
      empresa['produto'].append(produto)

      for x in municipios:
        for y in municipios:
          max_x = df[df['Municipio_dest'] == x].max()['Valor_unit_prod']
          min_y = df[df['Municipio_dest'] == y].min()['Valor_unit_prod']
          valor = max_x-min_y

          matriz[y][x] = valor

      empresa['matriz'].append(matriz)

  empresa = pd.DataFrame(data=empresa)
  empresa['indicador'] = empresa['matriz'].apply(lambda x: (np.absolute(x.values).sum())/(x.shape[0]**2))
  empresa['n_municipios'] = empresa['matriz'].apply(lambda x: x.shape[0])
  empresa = empresa.dropna(subset=['indicador'])
  empresa['peso'] = empresa['indicador']*empresa['valor_total']

  return empresa      

In [38]:
cenoura_ = sobrepreco_mun(cenoura,'cenoura')
tomate_ = sobrepreco_mun(tomate,'tomate')
cebola_ = sobrepreco_mun(cebola,'cebola')
alho_ = sobrepreco_mun(alho,'alho')
batata_ingl = sobrepreco_mun(batata_inglesa,'batata_inglesa')
cuscuz_ = sobrepreco_mun(cuscuz, 'cuscuz')

# Result of comparing the values of the sale of the same company to different counties

Price comparison of carrot sales

In [ ]:
cenoura_

,cnpj,periodo,matriz,valor_total,produto,indicador,n_municipios,peso
6,16759351000110,7-8,PATOS index PATOS 0.0,1131.68,cenoura,0.000000,1,0.000000
7,16759351000110,8-9,PATOS index PATOS 0.0,1740.64,cenoura,0.000000,1,0.000000
8,16759351000110,9-10,PATOS index PATOS 2.52,595.88,cenoura,2.520000,1,1501.617600
9,16759351000110,10-11,PATOS index PATOS 0.0,1077.12,cenoura,0.000000,1,0.000000
10,16759351000110,11-12,PATOS index PATOS 0.0,1805.76,cenoura,0.000000,1,0.000000
13,08792028000110,2-3,PICUI GUARABIRA ALAGOINHA LUCENA...,2505.97,cenoura,0.981250,4,2458.983062
14,08792028000110,3-4,PILOEZINHOS ALAGOINHA GUARABIRA...,4711.37,cenoura,0.906250,4,4269.679063
15,08792028000110,4-5,PILOEZINHOS GUARABIRA ALAGOINHA...,4147.87,cenoura,0.418750,4,1736.920563
16,08792028000110,5-6,PILOEZINHOS MULUNGU GUARABIRA ...,20733.52,cenoura,0.311111,6,6450.428444
17,08792028000110,6-7,GUARABIRA ALAGOINHA JOAO PESSOA...,24719.87,cenoura,0.200000,4,4943.974000


Matrix of counties with values of possible overprice indicators

In [ ]:
cenoura_["matriz"].values[8]

,PILOEZINHOS,MULUNGU,GUARABIRA,ALAGOINHA,JOAO PESSOA,LUCENA
index,,,,,,
PILOEZINHOS,0.00,0.30,0.65,0.45,0.65,0.85
MULUNGU,-0.30,0.00,0.35,0.15,0.35,0.55
GUARABIRA,-0.40,-0.10,0.25,0.05,0.25,0.45
ALAGOINHA,-0.40,-0.10,0.25,0.05,0.25,0.45
JOAO PESSOA,-0.65,-0.35,0.00,-0.20,0.00,0.20
LUCENA,-0.85,-0.55,-0.20,-0.40,-0.20,0.00


## Plots of the average difference in sales prices between counties

In [26]:
px.scatter(cuscuz_,x='indicador',y='n_municipios',hover_data=['cnpj','periodo','valor_total'],size='valor_total',color='cnpj',labels={
                     "n_municipios": "Number of counties",
                     "indicador": "Average difference in sales prices between counties (R$)",
                     'cnpj':'National Register of Legal Entity (CNPJ)'
                 }, title='Corn Flakes ({} R$ of median)'.format(cuscuz['Valor_unit_prod'].median()))

In [27]:
px.scatter(cenoura_,x='indicador',y='n_municipios',hover_data=['cnpj','periodo','valor_total'],size='valor_total',color='cnpj',labels={
                     "n_municipios": "Number  of Counties",
                     "indicador": "Indicator",
                     'cnpj':'National Register of Legal entity (CNPJ)'
                 }, title='Carrots')

In [28]:
px.scatter(tomate_,x='indicador',y='n_municipios',hover_data=['cnpj','periodo'],size='valor_total',color='cnpj',labels={
                     "n_municipios": "Number  of Counties",
                     "indicador": "Indicator",
                     'cnpj':'National Register of Legal entity (CNPJ)'
                 }, title='Tomato')

In [29]:
px.scatter(cebola_,x='indicador',y='n_municipios',hover_data=['cnpj','periodo'],size='valor_total',color='cnpj',labels={
                    "n_municipios": "Number  of Counties",
                     "indicador": "Indicator",
                     'cnpj':'National Register of Legal entity (CNPJ)'
                 }, title='Onoin')

In [30]:
px.scatter(alho_,x='indicador',y='n_municipios',hover_data=['cnpj','periodo'],size='valor_total',color='cnpj',labels={
                     "n_municipios": "Number  of Counties",
                     "indicador": "Indicator",
                     'cnpj':'National Register of Legal entity (CNPJ)'
                 }, title='Garlic')

In [31]:
px.scatter(batata_ingl,x='indicador',y='n_municipios',hover_data=['cnpj','periodo'],size='valor_total',color='cnpj',labels={
                     "n_municipios": "Number  of Counties",
                     "indicador": "Indicator",
                     'cnpj':'National Register of Legal entity (CNPJ)'
                 }, title='Potato')

# Result of price difference in the same counties

In [41]:
def sobrepreco(dataframe,cnpj):
  empresa = pd.DataFrame()
  periodos = []

  cnpj = dataframe['CPF_CNPJ_emit'].value_counts().to_frame()
  cnpj = cnpj.loc[cnpj['CPF_CNPJ_emit'] >= 180].index

  for n in cnpj:
    for i in range(1,13):
      start_date = datetime(year=2016, month=i, day=1).date()
      end_date = datetime(year=2016, month=i+1 if i < 12 else i, day=1).date()
      filtro_data = (dataframe['Data_de_emissao'] >= start_date) & (dataframe['Data_de_emissao'] < end_date)
          
      df = dataframe[(dataframe['CPF_CNPJ_emit']== n) & filtro_data]
            
      max = df.groupby(['Municipio_dest']).max()['Valor_unit_prod']
      min = df.groupby(['Municipio_dest']).min()['Valor_unit_prod']
      med = df.groupby(['Municipio_dest']).median()['Valor_unit_prod']
      dif = max-min
      porcentagem = (dif/min)*100

      transp = pd.DataFrame()
      transp = transp.append(dif).transpose()
      transp.columns=['difference']
      transp['period'] = str(str(i)+'-'+str(i+1))
      transp['company'] = n
      transp['percentage(max/min)'] = porcentagem
      
      empresa = empresa.append(transp)
    
  empresa = empresa.reset_index().rename(columns={'index':'county'})
  return empresa

In [42]:
cenoura = sobrepreco(dataframe=cenoura,cnpj=['08792028000110','16759351000110','24506743000104','09087805000199','00301402000126'])
tomate = sobrepreco(dataframe=tomate,cnpj=['16759351000110', '08792028000110','09087805000199','24506743000104','13407964000109'])
cebola = sobrepreco(dataframe=cebola,cnpj=['16759351000110','08792028000110','24506743000104','09087805000199','02775367000102'])
alho = sobrepreco(dataframe=alho,cnpj=['16759351000110','08792028000110','19074142000121','18107594000108','02927946000123'])
batata_inglesa = sobrepreco(dataframe=batata_inglesa,cnpj=['24506743000104','08792028000110','09087805000199','13407964000109','00301402000126'])

Price comparison of carrot sales

In [44]:
cenoura[cenoura['difference']!=0].sort_values('difference',ascending=False)

,county,difference,period,company,percentage(max/min)
10,GUARABIRA,2.55,3-4,08792028000110,137.837838
6,GUARABIRA,2.55,2-3,08792028000110,137.837838
28,GUARABIRA,2.55,7-8,08792028000110,137.837838
2,PATOS,2.52,9-10,16759351000110,252.000000
69,JOAO PESSOA,1.59,11-12,24506743000104,54.452055
5,ALAGOINHA,1.00,2-3,08792028000110,29.411765
55,JOAO PESSOA,0.80,4-5,24506743000104,23.880597
53,JOAO PESSOA,0.80,3-4,24506743000104,23.880597
44,GUARABIRA,0.80,11-12,08792028000110,18.181818
67,JOAO PESSOA,0.74,10-11,24506743000104,19.628647


Price comparison of tomato sales

In [45]:
tomate[tomate['difference']!=0].sort_values('difference',ascending=False)

,county,difference,period,company,percentage(max/min)
5,ALAGOINHA,1.15,2-3,08792028000110,31.081081
14,GUARABIRA,1.15,4-5,08792028000110,31.081081
6,GUARABIRA,1.05,2-3,08792028000110,30.000000
10,GUARABIRA,1.05,3-4,08792028000110,30.000000
28,GUARABIRA,1.05,7-8,08792028000110,30.000000
36,GUARABIRA,0.90,9-10,08792028000110,24.657534
33,GUARABIRA,0.90,8-9,08792028000110,24.657534
24,GUARABIRA,0.90,6-7,08792028000110,24.657534
18,GUARABIRA,0.90,5-6,08792028000110,24.657534
45,GUARABIRA,0.65,11-12,08792028000110,14.285714


Price comparison of onion sales

In [46]:
cebola[cebola['difference']!=0].sort_values('difference',ascending=False)

,county,difference,period,company,percentage(max/min)
6,GUARABIRA,2.40,2-3,08792028000110,114.285714
10,GUARABIRA,2.40,3-4,08792028000110,114.285714
28,GUARABIRA,2.40,7-8,08792028000110,114.285714
14,GUARABIRA,0.60,4-5,08792028000110,15.384615
18,GUARABIRA,0.35,5-6,08792028000110,8.433735
24,GUARABIRA,0.35,6-7,08792028000110,8.433735
33,GUARABIRA,0.35,8-9,08792028000110,8.433735
36,GUARABIRA,0.35,9-10,08792028000110,8.433735
45,GUARABIRA,0.30,11-12,08792028000110,6.666667


Price comparison of garlic sales

In [47]:
alho[alho['difference']!=0].sort_values('difference',ascending=False)

,county,difference,period,company,percentage(max/min)
43,GUARABIRA,9.5,11-12,08792028000110,61.290323
18,GUARABIRA,0.5,5-6,08792028000110,3.225806
23,GUARABIRA,0.5,6-7,08792028000110,3.225806
31,GUARABIRA,0.5,8-9,08792028000110,3.225806
34,GUARABIRA,0.5,9-10,08792028000110,3.225806


Price comparison of potato sales

In [48]:
batata_inglesa[batata_inglesa['difference']!=0].sort_values('difference',ascending=False)

,county,difference,period,company,percentage(max/min)
43,GUARABIRA,2.50,7-8,08792028000110,108.695652
21,GUARABIRA,2.50,2-3,08792028000110,108.695652
25,GUARABIRA,2.50,3-4,08792028000110,108.695652
19,JOAO PESSOA,2.29,11-12,24506743000104,85.130112
20,ALAGOINHA,1.40,2-3,08792028000110,41.176471
29,GUARABIRA,0.90,4-5,08792028000110,23.076923
9,JOAO PESSOA,0.78,6-7,24506743000104,18.571429
48,GUARABIRA,0.60,8-9,08792028000110,14.285714
39,GUARABIRA,0.60,6-7,08792028000110,14.285714
33,GUARABIRA,0.60,5-6,08792028000110,14.285714
